# Chapter 11: Markov chains

## Matrix Calculations

Let's do some calculations for the $4$-state Markov chain in Example 11.1.5, as an example of working with transition matrices in R. First, we need to specify the transition matrix $Q$. This is done with the `matrix` command: we type in the entries of the matrix, row by row, as a long vector, and then we tell R the number of rows and columns in the matrix (`nrow` and `ncol`), as well as the fact that we typed in the entries by row (`byrow=TRUE`):

In [ ]:
Q <- matrix(c(1/3,1/3,1/3,0,
    0,0,1/2,1/2,
    0,1,0,0,
    1/2,0,0,1/2),nrow=4,ncol=4,byrow=TRUE)

To obtain higher order transition probabilities, we can multiply $Q$ by itself repeatedly. The matrix multiplication command in R is `%*%` (_not_ just `*`). So

In [ ]:
Q2 <- Q %*% Q
Q3 <- Q2 %*% Q
Q4 <- Q2 %*% Q2
Q5 <- Q3 %*% Q2

produces $Q^2$ through $Q^5$. If we want to know the probability of going from state $3$ to state $4$ in exactly $5$ steps, we can extract the $(3,4)$ entry of $Q^5$:

In [ ]:
Q5[3,4]

This gives $0.229$, agreeing with the value $11/48$ shown in Example 11.1.5.

To compute a power $Q^n$ without directly doing repeated matrix multiplications, we can use the command `Q %^% n` after installing and loading the `expm` package. For example, `Q %^% 42` yields $Q^{42}$. By exploring the behavior of $Q^n$ as $n$ grows, we can see Theorem 11.3.6 in action (and get a sense of how long it takes for the chain to get very close to its stationary distribution).

In particular, for $n$ large each row of $Q^n$ is approximately $(0.214,0.286,0.214,0.286)$, so this is approximately the stationary distribution. Another way to obtain the stationary distribution numerically is to use

In [ ]:
eigen(t(Q))

to compute the eigenvalues and eigenvectors of the transpose of $Q$; then the eigenvector corresponding to the eigenvalue $1$ can be selected and normalized so that the components sum to $1$.

## Gambler's ruin

To simulate from the gambler's ruin chain from Example 11.2.6, we start by deciding the total amount of money `N`, the probability `p` of gambler A winning a given round, and the number of time periods `nsim` that we wish to simulate.

In [ ]:
N <- 10
p <- 1/2
nsim <- 80

Next, we allocate a vector of length `nsim` called `x`, which will store the values of the Markov chain. For the initial condition, we set the first entry of `x` equal to $5$; this gives both gamblers $5 to start with.

In [ ]:
x <- rep(0,nsim)
x[1] <- 5

Now we are ready to simulate the subsequent values of the Markov chain. This is achieved with the following block of code, which we will explain step by step.

In [ ]:
for (i in 2:nsim){
    if (x[i-1]==0 || x[i-1]==N){
        x[i] <- x[i-1]
    }
    else{
        x[i] <- x[i-1] + sample(c(1,-1), 1, prob=c(p,1-p))
    }
}

The first line and the outer set of braces constitute a _for loop_: `for (i in 2:nsim)` means that all the code inside the for loop will be executed over and over, with the value of `i` set to $2$, then set to $3$, then set to $4$, all the way until `i` reaches the value `nsim`. Each pass through the loop represents one step of the Markov chain.

Inside the for loop, we first check to see whether the chain is already at one of the endpoints, $0$ or `N`; we do this with an _if statement_. If the chain is already at $0$ or `N`, then we set its new value equal to its previous value, since the chain is not allowed to escape $0$ or `N`. Otherwise, if the chain is not at $0$ or `N`, it is free to move left or right. We use the `sample` command to move to the right 1 unit or to the left 1 unit, with probabilities `p` and `1-p`, respectively.

To see what path was taken by the Markov chain during our simulation, we can plot the values of `x` as a function of time:

In [ ]:
plot(x,type='l',ylim=c(0,N))

You should see a path that starts at $5$ and bounces up and down before being absorbed into state $0$ or state `N`.

## Simulating from a finite-state Markov chain

With a few modifications, we can simulate from an arbitrary Markov chain on a finite state space. For concreteness, we will illustrate how to simulate from the $4$-state Markov chain in Example 11.1.5.

As above, we can type

In [ ]:
Q <- matrix(c(1/3,1/3,1/3,0,
              0,0,1/2,1/2,
              0,1,0,0,
              1/2,0,0,1/2),nrow=4,ncol=4,byrow=TRUE)

to specify the transition matrix $Q$.

Next, we choose the number of states and the number of time periods to simulate, we allocate space for the results of the simulation, and we choose initial conditions for the chain. In this example, `x[1] <- sample(1:M,1)` says the initial distribution of the chain is uniform over all states.

In [ ]:
M <- nrow(Q)
nsim <- 10^4
x <- rep(0,nsim)
x[1] <- sample(1:M,1)

For the simulation itself, we again use `sample` to choose a number from $1$ to `M`. At time `i`, the chain was previously at state `x[i-1]`, so we must use row `x[i-1]` of the transition matrix to determine the probabilities of sampling $1,2,\ldots,$`M`. The notation `Q[x[i-1],]` denotes row `x[i-1]` of the matrix `Q`.

In [ ]:
for (i in 2:nsim){
    x[i] <- sample(M, 1, prob=Q[x[i-1],])
}

Since we set `nsim` to a large number, it may be reasonable to believe that the chain is close to stationarity during the latter portion of the simulation. To check this, we eliminate the first half of the simulations to give the chain time to reach stationarity:

In [ ]:
x <- x[-(1:(nsim/2))]

We then use the `table` command to calculate the number of times the chain visited each state; dividing by `length(x)` converts the counts into proportions. The result is an approximation to the stationary distribution.

In [ ]:
table(x)/length(x)

For comparison, the true stationary distribution of the chain is approximately $(0.214,0.286,0.214,0.286)$. Is this close to what you obtained via simulation?